In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

In [ ]:
soup = BeautifulSoup(source, 'html.parser')

In [ ]:
postal_dict = {}
for i in soup.find_all('td'):
    body = soup.body
for paragraph in body.find_all('p'):
    print(paragraph.text)
for div in soup.find_all('div', class_='body'):
    print(div.text)

In [ ]:
for i in soup.find_all('td'):
    try:
        postal_code = i.p.b.text 
        postal_code_text = i.span.text
        nb_data = table_cell.span.text 
        borough = nb_data.split('(')[0] 
        
        if nb_data == 'Not assigned':
            nb = []
        else:
            postal_codes_dict[postal_code] = {}
            
            try:
                nb = nb_data.split('(')[1]
            
                # remove parantheses from neighborhoods string
                nb = nb.replace('(', ' ')
                nb = nb.replace(')', ' ')

                nb_names = nb.split('/')
                nb_clean = ', '.join([name.strip() for name in nb_names])
            except:
                borough = borough.strip('\n')
                nb_clean = borough
 
            # add borough and neighborhood to dictionary
            postal_dict[postal_code]['borough'] = borough
            postal_dict[postal_code]['neighborhoods'] = nb_clean
    except:
        pass
    
columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns=columns)
toronto_data

# populate dataframe with data from dictionary
for i, j in enumerate(postal_dict):
    borough = postal_dict[postal_code]['borough']
    nb = postal_dict[postal_code]['neighborhoods']
    toronto_data = toronto_data.append({"PostalCode": postal_code, 
                                        "Borough": borough, 
                                        "Neighborhood": nb},
                                        ignore_index=True)

# print number of rows of dataframe
toronto_data.shape[0]

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]